# [Copy List with Random Pointer](https://leetcode.com/problems/copy-list-with-random-pointer/description/)

Clone a linked list where each node has a `.next` and a `.random` pointer.

## Strategy

### Using Hash


__Original:__
```
A → B → C
|    |    |
v    v    v
C    A    B
```
__Map:__
```
A → A'
B → B'
C → C'
```
__Then:__
```
A'.next = B'
A'.random = C'
...
```

1. Pass 1: Clone nodes and store mapping
- Traverse the original list.
- For each node, create a new node with the same `val`.
- Store a mapping from` original node → copied node` in a `Map<Node, Node>`.
This mapping will let us assign `next` and `random` in the second pass.
2. Pass 2: Assign next and random pointers
- Traverse the list again.
- For each original node:
    - `Set copy.next = map.get(original.next)`
    - `Set copy.random = map.get(original.random)`
3. Final Step
- Return `map.get(originalHead)` as the new deep-copied list head.

__Time__: O(n)  
__Space__: O(n) 
|
### Interleaved Linked List

1. First Pass:
    - Iterate through the original list.
    - For each node orig, create its copy and insert it right after orig (so the list becomes: orig -> copy -> orig.next -> copy -> ...).
    - This helps you keep the mapping between original and copy without extra memory.
2. Second Pass:
    - Now that orig.next is always its copy, use that to assign the random pointer of the copy using:
`copy.random = orig.random ? orig.random.next : nullptr`
3. Third Pass:
    - Unweave the combined list: separate the original and copied nodes into two distinct linked lists.

__Time__: O(n)  
__Space__: O(1)  

## Solution


In [1]:
import { RandomListNode } from "../bin/ds/list.ts"

/**
 * Definition for singly-linked list.
 * class ListNode {
 *     val: number
 *     next: ListNode | null
 *     constructor(val?: number, next?: ListNode | null) {
 *         this.val = (val===undefined ? 0 : val)
 *         this.next = (next===undefined ? null : next)
 *     }
 * }
 */

// Time: O(n)
// Space: O(n)
function copyRandomList(head: RandomListNode | null): RandomListNode | null {
    const map: Map<_Node, _Node> = new Map()
    for(let curr = head; curr !== null; curr = curr.next) {
        const copy = new RandomListNode(curr.val)
        map.set(curr, copy)
    }
    for(let curr = head; curr !== null; curr = curr.next) {
        const copy = map.get(curr)!
        copy.next = map.get(curr.next) ?? null;
        copy.random = map.get(curr.random) ?? null;
    }

    return map.get(head) ?? null
};

// Time: O(n)
// Space: O(1)
function copyRandomList_interleaved(head: RandomListNode | null): RandomListNode | null {
    let curr: RandomListNode | null = head
    while(curr) {
        const copy = new RandomListNode(curr.val)
        copy.next = curr.next
        curr.next = copy 
        curr = copy.next
    }
    curr = head
    while(curr) {
        const copy: RandomListNode | null = curr.next
        copy.random = curr.random?.next
        curr = curr?.next?.next
    }
    curr = head
    let copy: RandomListNode | null = head?.next
    let copyHead = copy

    while (curr && copy) {
        curr.next = copy.next
        curr = curr.next

        if (curr) {
            copy.next = curr.next
            copy = copy.next
        }
    }

    return copyHead
};

In [2]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("copyRandomList - two node cycle", () => {
  const head = RandomListNode.fromArray([
    [1, 1],
    [2, 0]
  ]);
  const copy = copyRandomList(head);
  assertEquals(copy?.toArray(), [
    [1, 1],
    [2, 0]
  ]);
});

Deno.test("copyRandomList - multiple null randoms", () => {
  const head = RandomListNode.fromArray([
    [3, null],
    [4, null],
    [5, null]
  ]);
  const copy = copyRandomList(head);
  assertEquals(copy?.toArray(), [
    [3, null],
    [4, null],
    [5, null]
  ]);
});

copyRandomList - two node cycle ... ok (1ms)
copyRandomList - multiple null randoms ... ok (0ms)

ok | 2 passed | 0 failed (1ms)
